In [ ]:
#https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html#sphx-glr-auto-examples-sklearn-gridsearchcv-replacement-py
#https://skopt.readthedocs.io/en/stable/install.html
#https://tex.stackexchange.com/questions/480652/prevent-table-from-going-off-page-how
#install and import packages
!pip install --upgrade scikit-learn
!pip install pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

#suppress warnings about class imbalances
import warnings
warnings.filterwarnings("ignore")

#import models, packages
from sklearn import linear_model, ensemble
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
import numpy

!pip install scikit-optimize
import skopt
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV

#import grid search and cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
#print python version for report
!python ‐‐version
import sys; print(sys.version)

python3: can't open file '/content/‐‐version': [Errno 2] No such file or directory
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
#read and display data
data = pd.read_csv("/content/winequality-red.csv", sep = ";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [ ]:
#display information about dataset for report
data.columns
print(data.groupby(['quality']).count())

         fixed acidity  volatile acidity  citric acid  residual sugar  \
quality                                                                 
3                   10                10           10              10   
4                   53                53           53              53   
5                  681               681          681             681   
6                  638               638          638             638   
7                  199               199          199             199   
8                   18                18           18              18   

         chlorides  free sulfur dioxide  total sulfur dioxide  density   pH  \
quality                                                                       
3               10                   10                    10       10   10   
4               53                   53                    53       53   53   
5              681                  681                   681      681  681   
6              638  

In [ ]:
#split into features/target
x = data.drop(columns = ['quality'])
#attemptd normalization at one point but this step yielded lower performance
#x_norm = sklearn.preprocessing.normalize(x, axis=0)
y = data['quality']

In [ ]:
ridge_opt = BayesSearchCV(linear_model.RidgeClassifier(),
    {
    "alpha": Real(1.0, 5.0, prior='log-uniform'),
    "tol": Real(0.0001, 0.1, prior='log-uniform'),
    "solver": Categorical(["svd", "cholesky", "lsqr", "sparse_cg"]),
    "max_iter": Integer(100, 100000, prior='log-uniform'),
    },
     n_iter=32,
     random_state=0,
     scoring = "balanced_accuracy"

)

bagging_opt = BayesSearchCV(ensemble.BaggingClassifier(),
                {"n_estimators" : Integer(100, 10000, prior = 'log-uniform'),
               "max_samples" : Real(0.01, 1.0, prior='log-uniform'),
                "max_features" : Real(0.01, 1.0, prior='log-uniform'),
                },
        n_iter=32,
        random_state=0,
        scoring = "balanced_accuracy"

)

rf_opt = BayesSearchCV(ensemble.RandomForestClassifier(),
        {
    "n_estimators" : Integer(100, 10000, prior = 'log-uniform'),
    "criterion" : Categorical(["gini", "entropy", "log_loss"]),
    "max_depth" : Real(2, 10, prior='log-uniform')
        },
        n_iter=32,
        random_state=0,
        scoring = "balanced_accuracy"
)

In [ ]:
def nested_resampling_bayesian(optimizer, x, y):
     for i in range(0, 3): #3-fold outer cross-validation
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=42)
        #10_fold inner resampling -- performs 10-fold cross-validation on each hyperparameter configuration and stores results
        cv_results = cross_validate(
           optimizer, x_train, y_train, cv=10, return_estimator=True, scoring = "balanced_accuracy"
        )
        cv_results = pd.DataFrame(cv_results)
        print(cv_results)
        cv_test_scores = cv_results["test_score"]
        #display results
        print(
            "Generalization score with hyperparameters tuning:\n"
            f"{cv_test_scores.mean():.3f} ± {cv_test_scores.std():.3f}"
        )
        optimizer.fit(x_train, y_train)
        print(optimizer.score(x_test, y_test))
        print(optimizer.best_estimator_)
        print(optimizer.best_params_)
        print(optimizer.best_score_)


In [ ]:
nested_resampling_bayesian(ridge_opt, x, y)

    fit_time  score_time                                          estimator  \
0  45.092385    0.004227  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
1  45.898100    0.011164  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
2  41.935857    0.004243  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
3  40.463408    0.004167  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
4  38.543196    0.003902  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
5  49.101809    0.003930  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
6  40.639992    0.004154  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
7  41.628239    0.004030  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
8  45.194503    0.003898  BayesSearchCV(estimator=RidgeClassifier(), n_i...   
9  40.173183    0.004084  BayesSearchCV(estimator=RidgeClassifier(), n_i...   

   test_score  
0    0.212240  
1    0.215508  
2    0.221331  
3    0.221331  
4    0.253298  
5    0.227726  
6    0.231310  
7 

In [ ]:
nested_resampling_bayesian(bagging_opt, x, y)

In [ ]:
nested_resampling_bayesian(rf_opt, x, y)